<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#File-paths" data-toc-modified-id="File-paths-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>File paths</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Creación-del-Diccionario" data-toc-modified-id="Creación-del-Diccionario-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Creación del Diccionario</a></span></li><li><span><a href="#BOW-generation" data-toc-modified-id="BOW-generation-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>BOW generation</a></span></li><li><span><a href="#Similarity-matrix-generation" data-toc-modified-id="Similarity-matrix-generation-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Similarity matrix generation</a></span></li></ul></li><li><span><a href="#Data-load" data-toc-modified-id="Data-load-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data load</a></span><ul class="toc-item"><li><span><a href="#Titles" data-toc-modified-id="Titles-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Titles</a></span></li><li><span><a href="#Judgements" data-toc-modified-id="Judgements-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Judgements</a></span></li><li><span><a href="#Queries" data-toc-modified-id="Queries-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Queries</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Metadata</a></span></li></ul></li><li><span><a href="#Retrieval-models" data-toc-modified-id="Retrieval-models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Retrieval models</a></span><ul class="toc-item"><li><span><a href="#TFIDF-Model" data-toc-modified-id="TFIDF-Model-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>TFIDF Model</a></span><ul class="toc-item"><li><span><a href="#Example-of-evaluation" data-toc-modified-id="Example-of-evaluation-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Example of evaluation</a></span></li><li><span><a href="#Parameterized-Evaluation" data-toc-modified-id="Parameterized-Evaluation-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Parameterized Evaluation</a></span></li></ul></li><li><span><a href="#TFIDF-2.0" data-toc-modified-id="TFIDF-2.0-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>TFIDF 2.0</a></span></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Word2Vec</a></span><ul class="toc-item"><li><span><a href="#Creation" data-toc-modified-id="Creation-6.3.1"><span class="toc-item-num">6.3.1&nbsp;&nbsp;</span>Creation</a></span></li><li><span><a href="#Saving" data-toc-modified-id="Saving-6.3.2"><span class="toc-item-num">6.3.2&nbsp;&nbsp;</span>Saving</a></span></li><li><span><a href="#Loading" data-toc-modified-id="Loading-6.3.3"><span class="toc-item-num">6.3.3&nbsp;&nbsp;</span>Loading</a></span></li><li><span><a href="#Getting-embedding" data-toc-modified-id="Getting-embedding-6.3.4"><span class="toc-item-num">6.3.4&nbsp;&nbsp;</span>Getting embedding</a></span></li><li><span><a href="#Queries-embeddings" data-toc-modified-id="Queries-embeddings-6.3.5"><span class="toc-item-num">6.3.5&nbsp;&nbsp;</span>Queries embeddings</a></span></li><li><span><a href="#Documents-embeddings" data-toc-modified-id="Documents-embeddings-6.3.6"><span class="toc-item-num">6.3.6&nbsp;&nbsp;</span>Documents embeddings</a></span></li><li><span><a href="#Launch-query-in-W2V-Model" data-toc-modified-id="Launch-query-in-W2V-Model-6.3.7"><span class="toc-item-num">6.3.7&nbsp;&nbsp;</span>Launch query in W2V Model</a></span></li></ul></li><li><span><a href="#Fast-text" data-toc-modified-id="Fast-text-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Fast text</a></span><ul class="toc-item"><li><span><a href="#Creation" data-toc-modified-id="Creation-6.4.1"><span class="toc-item-num">6.4.1&nbsp;&nbsp;</span>Creation</a></span></li><li><span><a href="#Saving" data-toc-modified-id="Saving-6.4.2"><span class="toc-item-num">6.4.2&nbsp;&nbsp;</span>Saving</a></span></li><li><span><a href="#Loading" data-toc-modified-id="Loading-6.4.3"><span class="toc-item-num">6.4.3&nbsp;&nbsp;</span>Loading</a></span></li><li><span><a href="#Getting-embeddings" data-toc-modified-id="Getting-embeddings-6.4.4"><span class="toc-item-num">6.4.4&nbsp;&nbsp;</span>Getting embeddings</a></span></li><li><span><a href="#Queries-embeddings" data-toc-modified-id="Queries-embeddings-6.4.5"><span class="toc-item-num">6.4.5&nbsp;&nbsp;</span>Queries embeddings</a></span></li><li><span><a href="#Documents-embeddings" data-toc-modified-id="Documents-embeddings-6.4.6"><span class="toc-item-num">6.4.6&nbsp;&nbsp;</span>Documents embeddings</a></span></li><li><span><a href="#Launch-query-in-FastText-Model" data-toc-modified-id="Launch-query-in-FastText-Model-6.4.7"><span class="toc-item-num">6.4.7&nbsp;&nbsp;</span>Launch query in FastText Model</a></span></li></ul></li><li><span><a href="#Latent-Semantic-Indexing" data-toc-modified-id="Latent-Semantic-Indexing-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Latent Semantic Indexing</a></span><ul class="toc-item"><li><span><a href="#Model-creation" data-toc-modified-id="Model-creation-6.5.1"><span class="toc-item-num">6.5.1&nbsp;&nbsp;</span>Model creation</a></span></li><li><span><a href="#Similarities-matrix" data-toc-modified-id="Similarities-matrix-6.5.2"><span class="toc-item-num">6.5.2&nbsp;&nbsp;</span>Similarities matrix</a></span></li><li><span><a href="#Launching-queries" data-toc-modified-id="Launching-queries-6.5.3"><span class="toc-item-num">6.5.3&nbsp;&nbsp;</span>Launching queries</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Conclusions</a></span></li><li><span><a href="#References" data-toc-modified-id="References-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Prerequisites

Make sure you have downloaded the following pakcages in your environment:

- nltk: nltk.download('stopwords')
- xmltodict
- numpy
- gensim
- pandas
- matplotlib
- dask
- sklearn

In [1]:
pip install nltk

  Using cached nltk-3.5-py3-none-any.whl
  Using cached joblib-0.17.0-py3-none-any.whl (301 kB)
  Using cached regex-2020.11.13-cp38-cp38-win_amd64.whl (270 kB)
  Using cached tqdm-4.54.1-py2.py3-none-any.whl (69 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xmltodict

  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --user numpy==1.19.3

  Using cached numpy-1.19.3-cp38-cp38-win_amd64.whl (13.3 MB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 1.0.1.post1 requires pyyaml, which is not installed.
ray 1.0.1.post1 requires requests, which is not installed.


In [4]:
pip install gensim

  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)
  Using cached Cython-0.29.14-cp38-cp38-win_amd64.whl (1.7 MB)
  Using cached scipy-1.5.4-cp38-cp38-win_amd64.whl (31.4 MB)
  Using cached smart_open-4.0.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

  Using cached pandas-1.1.5-cp38-cp38-win_amd64.whl (9.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.23.0 requires requests<3.0.0dev,>=2.18.0, which is not installed.
modin 0.8.2 requires pandas==1.1.4, but you have pandas 1.1.5 which is incompatible.


  Using cached pytz-2020.4-py2.py3-none-any.whl (509 kB)


In [6]:
pip install matplotlib

  Using cached matplotlib-3.3.3-cp38-cp38-win_amd64.whl (8.5 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp38-cp38-win_amd64.whl (51 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached Pillow-8.0.1-cp38-cp38-win_amd64.whl (2.1 MB)


In [7]:
pip install dask

  Using cached dask-2020.12.0-py3-none-any.whl (884 kB)
  Using cached PyYAML-5.3.1-cp38-cp38-win_amd64.whl (219 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 1.0.1.post1 requires requests, which is not installed.


In [8]:
pip install sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.2-cp38-cp38-win_amd64.whl (6.8 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David.Rubio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Imports

Import the required libraries for this project.

In [10]:
import json
import xmltodict as xtd
import numpy as np
import os
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer
from joblib import Parallel, delayed
from gensim import corpora
from gensim import models
import pandas as pd
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity
from operator import itemgetter
import glob
from matplotlib import pyplot as plt
import re

# File paths

Add the paths for the queries file, the texts (documents) and the judements file.

In [11]:
"""path_queries = r"B:\document_parser\document_parses\topics-rnd5.xml"
path_texts = "B:\document_parser\document_parses\pdf_json"
path_test = "B:/document_parser/document_parses/test"
path_judgements = "B:/document_parser/document_parses/judgements.csv"
path_judgements2 = "B:/document_parser/document_parses/judgements2.csv"
"""
path_queries = r"C:/Users/David.Rubio/Desktop/muia/topics-rnd5.xml"
path_texts = "C:/Users/David.Rubio/Desktop/muia/2020-07-16/document_parses/pdf_json"
path_judgements = "C:/Users/David.Rubio/Desktop/muia/qrels-covid_d5_j0.5-5.txt"
path_judgements2 = "C:/Users/David.Rubio/Desktop/muia/qrels-covid_d5_j0.5-5.txt"

# Preprocessing

In this section, the dictionary, the bag of words (bow) and the similarity matrix will be generated.

## Dictionary generation

Regarding the dictionary, we found out that there exists another special dictionary called HashDictionary, that doesn't need to be filled before used it. However, it is likely that two different words get the same key. So, we decided to use the basic one in spite of having to fill it before hand.

In [ ]:
# 2-5 minutes
dictionary = corpora.Dictionary(line.split() for line in open("docs.txt","r"))
dictionary.save('covid19.dict')  # store the dictionary, for future reference
print(dictionary)

## BOW generation

First, the stopwords list needs to be assigned to a variable. This list contains the most common words in a language (e.g. 'in', 'the', 'a'...).

In [ ]:
stopset = set(stopwords.words("english"))

The method *preprocess_document* processes a document given a stopword list. In this method, we concatenate the information given by the title, the abstract and the body of he document. These data are the most relevant for making a ranking given a query.

In [ ]:
def preprocess_document(doc,stopset):
    title = np.array([doc["metadata"]["title"]], dtype=str)
    abstract = np.array([paragraph["text"] for paragraph in doc["abstract"]], dtype=str)
    text = np.array([paragraph["text"] for paragraph in doc["body_text"]], dtype=str)
    stemmer = PorterStemmer()
    information = np.concatenate((title, abstract, text))
    tokens = np.concatenate(([wordpunct_tokenize(inf) for inf in information]))
    clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2 and not token.isnumeric()]
    final = " ".join([stemmer.stem(word) for word in clean])
    return title[0], str(final.encode("utf-8"))

With the data each file in path_texts we write one row per document in docs.txt with the most relevant information (use preprocess_document). This information will be essential to build the dictionary. This step takes more than an hour so, unless you need to generate the file, we strongly suggest not to remove the comments.

In [ ]:
# More than 1 hour
files = glob.glob(path_texts + "/*.json")
with open("docs.txt", "w+") as f:
    i = 1
    for file in files:
        print(i)
        i+=1
        with open(file) as js:
            file_json = json.load(js)
        _, stems = preprocess_document(file_json, stopset)
        f.write(stems.strip('b\'')+"\n")

The class MyCorpus will help to iterate (\_\_iter\_\_ method) over each line of a document.

In [ ]:
class MyCorpus:
    
    def __init__(self, file):
        self.path = file
        
    def __iter__(self):
        for line in open(self.path):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.split())

We iterate over each row in "docs.txt" to build the bag of words.

In [ ]:
# 2-5 minutes
path_corpus = "docs.txt"
bow = MyCorpus(path_corpus)
corpora.MmCorpus.serialize("covid19.mm", bow, metadata=True)
print(bow)

## Similarity matrix generation

Regarding the similarities, we chose to use Similarity instead of MatrixSimilarity since it allows lazy generation, not making us to store all the data in RAM.

In [ ]:
# 3-5 minutes
index = Similarity(None, corpus = bow, num_features=len(dictionary))  # create index
index.save("covid19.index")

# Data load 

In this section, relevant information will be loaded locally (title, judgmenets...). This information is useful during the evaluation step.

## Titles

The title and the paper_id for each document in "path_texts" are needed to link the paper_id retrieved by the model with the title.

For this task, dask is used to reduce the computing time (parallel computing): 6 workers with 2 threads per worker.

In [ ]:
import dask.bag as db
from dask.distributed import Client, progress
client = Client(n_workers=6, threads_per_worker=2)

client

The method load_dataset calls load_json for each file in the path (document). For this document, load_json calls load_dataset to get the paper_id, the title, the abstract and the body of the document.

In [ ]:
def load_dataset(path):
    files = glob.glob(path + "/*.json")
    b = db.from_sequence(files).map(load_json)
    df = b.to_dataframe(columns=["id", "title", "abstract", "body"])
    return df

def load_document(doc):
    title = doc["metadata"]["title"]
    abstract = " ".join([paragraph["text"] for paragraph in doc["abstract"]])
    text = " ".join([paragraph["text"] for paragraph in doc["body_text"]])
    returned = {}
    returned["id"] = doc["paper_id"]
    returned["title"] = title
    returned["abstract"] = abstract
    returned["body"] = text
    return returned

def load_json(file_path):
    with open(file_path) as file:
        file_json = json.load(file)
    returned = load_document(file_json)
    return returned

The dataset is built (dataframe with paper_id, title, abstract and body columns for each document in "path_texts"). Then, the relevant parts of each row are selected (id and title).

In [ ]:
dataset = load_dataset(path_texts)

In [ ]:
# 2-3 minutes
titles = dataset[["id","title"]].compute()

In [ ]:
titles

## Judgements

The next file needed for the evaluation is the judgements. This file contains for each query a set of cord_uid (unique identifiers for each paper) and, linked with each pair (query, cord_uid), the score (relevance). The score can be 0,1 or 2 so, the first step is to binarize it (1 if the score is greater or equal to 1, and 0 otherwise). The documents selected for each query comes from the suggestions given by different search engines. With these documents, a "pool" is generated and experts analyze if the documents in the pool are relevant for the query or not. The content of judgements will be the key to evaluate the performance of the model.

In [ ]:
def load_judgements(path_judgements):
    judgements = pd.read_csv(path_judgements, delimiter=' ', names = ["query", "cord_uid", "score"], usecols=[0,2,3])
    judgements.loc[judgements['score'] < 1, 'binary_score'] = 0
    judgements.loc[judgements['score'] >=1 , 'binary_score'] = 1
    return judgements

judgements = load_judgements(path_judgements2)
judgements

## Queries

The content of the file help to link the number of the query in the judgements file with the text of the query.

In [ ]:
def load_queries(queries_path):
    """
    Receives the path of the queries files and returns a dictionary containing all the queries.

    Parameters
    ----------
    queries_path : path of the queries file

    Returns
    -------
    dic_judgements : dictionary

    """
    with open(queries_path, "r") as xml_file:
        data_dict = xtd.parse(xml_file.read())
    xml_file.close()

    dic_queries = {}
    for query in data_dict["topics"]["topic"]:
        dic_queries[query["@number"]] = query["query"]

    df = pd.DataFrame.from_dict(dic_queries, orient='index', columns=['query'])

    return df

queries = load_queries(path_queries)
queries.head()

## Metadata

The metadata file link the identifier of the paper used for indexing the documents with the cord_uid given in the judgements file.

In [ ]:
#path_metadata = "B:/document_parser/document_parses/metadata.csv"
path_metadata = "C:/Users/David.Rubio/Desktop/muia/2020-07-16/metadata.csv"

In [ ]:
metadata = pd.read_csv(path_metadata, header = 0, usecols = [0,1])

In [ ]:
metadata = metadata.assign(sha=metadata.sha.str.split('; ')).explode('sha')
metadata

In [ ]:
metadata['sha'].isnull().values.sum()

In [ ]:
metadata = metadata.dropna()

# Retrieval models

## TFIDF Model

The TFIDF (Term Frequency Inversed Document Frequency) model is a numerical statistical model. This model returns a score which indicates how important is a word/query for a document in a corpus (collection). This score is proportional to the number of times the word(s) appears (tf) and inversely proportional to the number of documents that contains this word(s).

The first thing that we need to do is loading the bag of words created before.

In [ ]:
bow = corpora.MmCorpus('covid19.mm')

Then, we also load the dictionary and the similarity matrix.

In [ ]:
dictionary = corpora.Dictionary.load("covid19.dict")

In [ ]:
similarities = Similarity.load("covid19.index")

Once we loaded the bow, dictionary and similarities, we are able to create our TFIDF model.

In [ ]:
# 1-2 minutes
# Default model uses nfc from SMART notation.
model_tfidf = models.TfidfModel(bow)

We create a function to execute a query that returns the ranking of documents retrieved for that specific query. Moreover, a util function to preprocess the query is created:

In [ ]:
def launch_query_tfidf(model, dictionary, bow, index, query, titles, verbose = 0):
    """
    Given a specific query, it returns the ranking of documents.
    
    Parameters
    ----------
    model : tfidf model
    dictionary: dictionary created
    bow: bag of words
    index: similarities matrix
    query: specific query
    titles: dataframe of the titles
    verbose: flag for printing messages
    
    Returns
    -------
    ranking: ranking for the query in this format: [(doc_position, score), (doc_position, score), ... ]
    similarities: similarities matrix

    """
    stopset = set(stopwords.words("english"))
    index = index
    pq = preprocess_query_tfidf(query, stopset)
    vq = dictionary.doc2bow(pq)
    qtfidf = model[vq]
    sim = index[qtfidf]
    ranking = sorted(enumerate(sim), key=itemgetter(1), reverse=True)
    if verbose:
        print("Query ==> "+query)
        for doc, score in ranking[:5]:
            print("[ Score = " + "%.3f" % round(score,3) + " ] " + titles['title'].iloc[doc])
    return ranking, sim
        
def preprocess_query_tfidf(query, stopset):
    """
    Basic function that preprocess a query given a stopset: tokenization, lower case, stopwords removal and stemming.
    
    Parameters
    ----------
    query: query to preprocess
    stopset: set containing stopwords
    
    Returns
    -------
    stems: stems of the given query
    
    """
    stemmer = PorterStemmer()
    tokens = wordpunct_tokenize(query)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2]
    stems = [stemmer.stem(word) for word in clean]
    return stems

We execute it to show you how it works. (Notice that in this case verbose is marked as 1)

In [ ]:
ranking, sim = launch_query_tfidf(model_tfidf, dictionary, bow, similarities, queries.iloc[1][0], titles, verbose = 1)

### Example of evaluation

Once we are able to launch queries, it's time for evaluation. We are going to show you step by step the process of evaluation for the query launched before. Once this process is explained, we will create parameterized functions in order to improve the quality and organization of our code, making us able to reuse it later.

The first thing that we do is creating a dataframe from the ranking obtained.

In [ ]:
ranking = [[position[0], position[1]] for position in ranking]
ranking = pd.DataFrame(ranking, columns=["doc_position", "rel_score"])
ranking

Now, we include the sha identifier of each document result and its global ranking.

In [ ]:
results = ranking
results["sha"] = results["doc_position"].map(lambda x: titles["id"].iloc[x])
results["ranking"] = results.index
results

Now, we join this dataframe with metadata in order to obtain the cord_uid of each document retrieved. This will allow us later to obtain the relevance scores.

In [ ]:
results = results.set_index("sha").join(metadata.set_index("sha"))
results = results.sort_values(by=["ranking"])
results

We check that there are some null values in the cord_uid column. This means that some documents of our corpus don't have a cord_uid in the metadata, so we drop them.

In [ ]:
results['cord_uid'].isnull().sum()

In [ ]:
results = results.dropna()
results

Before, we launched query 2, so now we need to obtain the relevance judgements of query 2.

In [ ]:
judgements_parcial = judgements[judgements["query"] == 2]
judgements_parcial

Once we have the judgements of query 2, we join this judgements with our ranking. This operation will allow us to know for each document retrieved whether it was relevant or not.

In [ ]:
statistics = results.set_index("cord_uid").join(judgements_parcial.set_index("cord_uid"))
statistics

Again, we observe that there are null values. These null values mean that there are some documents of our corpus that are not relevance judged. 

At this point have two possible options:
* Drop the null values: since those documents are not judged, we don't know whether they are relevant or not for that query. It makes sense to us dropping those documents, since assuming that they are relevant or not introduces uncertainty in our model.
* Mark them as non-relevant: after reading the TREC guide for evaluation, we understood how these relevance judgements files are created by means of pools. It is indicated that documents not included in the pool should be marked as non-relevant. However, it's also said that this approach is a bit controversial. 

After some more research, we realized that this is one of the most controversial aspects on evaluating information retrieval systems. Finally, we opted for the first action, since it made more sense for us.

In [ ]:
# Drop values
statistics = statistics.dropna()
# Mark them as non relevant
#a["binary_score"].fillna(0, inplace=True)
statistics

At this point, we already know whether each retrieved document is relevant or not.

Now, we compute the total number of relevant documents, it will be used later.

And, we remove all the results whose computed tfidf score is 0.

In [ ]:
r = statistics["binary_score"].sum()
statistics = statistics[statistics["rel_score"]>0]

In [ ]:
statistics

We order our statistics by the ranking and reset the index.

We compute the actual rank (starting in 1).

In [ ]:
statistics = statistics.sort_values(by=["ranking"])
statistics = statistics.reset_index()
statistics["rank"] = statistics.index+1

Now, we compute precision and recall for each position of the ranking. Notice that we are making the table explained in the lecture notes (slide 51).

In [ ]:
statistics["relevant"] = statistics["binary_score"]
statistics["rel_retrieved"] = statistics["binary_score"].cumsum()
statistics["precision"] = statistics["rel_retrieved"] / statistics["rank"]
statistics["recall"] = statistics["rel_retrieved"] / r
statistics.drop(["doc_position", "score", "binary_score"], axis=1, inplace=True)

In [ ]:
statistics

We also compute the Mean Average Precision, the P@5 and P@10 and the R-Precision.

These metrics and the precision and recall will be very helpful for the evaluation of our models.

* **Precision** = relevant documents retrieved / documents retrieved. 
* **Recall** = relevant documents retrieved / total amount of relevant documents
* **P@X**: precision at position X of our ranking
* **R-Precision**: precision at position R of our ranking, being R the total number of relevant documents for that query (remember that before we computed some r? That r is this R.)
* **Mean Average Precision**: average of precisions after some relevant document is retrieved.

In [ ]:
mavp = (statistics["precision"] * statistics["relevant"]).sum() / r
r_precision = statistics.iloc[int(r-1)].precision
p_5 = statistics.iloc[4].precision
p_10 = statistics.iloc[9].precision

print(f"Mean average precision of TFIDF model for query 2 ==> {mavp}")
print(f"R-precision of TFIDF model for query 2 ==> {r_precision}")
print(f"P@5 of this model for query 2 ==> {p_5}")
print(f"P@10 of this model for query 2 ==> {p_10}")

Now we will plot the precision vs recall figure, but using the standarized metric as explained in class.

In [ ]:
precision = statistics["precision"].values
recall = statistics["recall"].values

In [ ]:
def get_higher_precision(statistics, recall):
    """
    Auxiliary function to obtain the higher precision for a given recall value.
    
    """
    try:
        index = statistics.index[statistics['recall'] >= recall].tolist()[0] 
        return statistics.iloc[index:]["precision"].max()
    except:
        return 0

In [ ]:
precision_recall = [[recall_i, get_higher_precision(statistics, recall_i)] for recall_i in np.arange(0,recall.max()+0.1, 0.1)]
precision = [precision[1] for precision in precision_recall]
recall = [recall[0] for recall in precision_recall]

In [ ]:
plt.figure(figsize=(5,5))
plt.title("Precision vs recall standardized")
plt.plot(recall, precision, "r-o")
plt.axis([0, 1, 0, 1])
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()

### Parameterized Evaluation

Before, we have explained step by step how the statistics are computed.

The following three cells have the same code as above, but in functions so as to be reused later.

In [ ]:
def launch(function, model,dictionary,bow,index,titles):
    """
    God function that allow us to pass independent launch query functions to our get_statistics function. 
    For more info, check Currying (Functional Programming).
    
    """
    def h(query):
        return function(model,dictionary,bow,index,query,titles)
    return h

In [ ]:
def get_statistics(launch_query, queries, titles, metadata, judgements):
    """
    General function that allow us to get statistics for all queries given a specific launch query function (model).
    
    Returns
    -------
    It returns all the metrics explained before.
    """
    results = []
    for index in range(len(queries)):
        ranking, _ = launch_query(queries.iloc[index][0])#
        result = get_statistics_query(ranking, index+1, titles, metadata, judgements)
        results.append(result)
    mavp = np.array([query["mavp"] for query in results]).mean()
    precision = np.array([query["precision"] for query in results]).mean(axis=0)
    r_precision = [query["r-precision"] for query in results]
    p_5 = [query["p_5"] for query in results]
    p_10 = [query["p_10"] for query in results]
    recall = np.array([query["recall"] for query in results]).mean(axis=0)
    return {"mavp": mavp, "precision":precision, "r-precision":r_precision, "recall":recall, "p_5": p_5, "p_10": p_10}

In [ ]:
def get_statistics_query(ranking, id_query, titles, metadata, judgements):
    """
    Individual function to compute the statistics for a specific query.
    
    Returns
    -------
    It returns all the metrics explained before.
    """
    ranking = [[position[0], position[1]] for position in ranking]
    ranking = pd.DataFrame(ranking, columns=["doc_position", "rel_score"])
    results = ranking
    results["sha"] = results["doc_position"].map(lambda x: titles["id"].iloc[x])
    results["ranking"] = results.index
    results = results.set_index("sha").join(metadata.set_index("sha"))
    results = results.sort_values(by=["ranking"])
    results = results.dropna()
    judgements_parcial = judgements[judgements["query"] == id_query]
    statistics = results.set_index("cord_uid").join(judgements_parcial.set_index("cord_uid"))
    statistics = statistics.dropna()
    r = statistics["binary_score"].sum()
    statistics = statistics[statistics["rel_score"]>0]
    statistics = statistics.sort_values(by=["ranking"])
    statistics = statistics.reset_index()
    statistics["rank"] = statistics.index+1
    statistics["relevant"] = statistics["binary_score"]
    statistics["rel_retrieved"] = statistics["binary_score"].cumsum()
    statistics["precision"] = statistics["rel_retrieved"] / statistics["rank"]
    r = statistics["relevant"].sum()
    if r == 0:
        return {"mavp": 0, "r-precision": 0, "precision": list(np.zeros(11)), "recall":0, "p_10": 0, "p_5": 0}
    statistics["recall"] = statistics["rel_retrieved"] / r
    statistics.drop(["doc_position", "score", "binary_score"], axis=1, inplace=True)
    mavp = (statistics["precision"] * statistics["relevant"]).sum() / r
    r_precision = statistics.iloc[int(r-1)].precision
    precision = statistics["precision"].values
    recall = statistics["recall"].values
    recall_range = np.arange(0,recall.max()+0.1, 0.1)
    precision_recall = [[recall, get_higher_precision(statistics, recall)] for recall in recall_range]
    precision = [precision[1] for precision in precision_recall]
    p_5 = statistics.iloc[4].precision
    p_10 = statistics.iloc[9].precision
    return {"mavp": mavp, "r-precision": r_precision, "precision": precision, "recall":recall_range, "p_10": p_10, "p_5":p_5}

## TFIDF 2.0

Once we have all our structure created: launching queries and computing statistics, we create 4 variations of the TFIDF Model playing with the SMART notation values.

In [ ]:
# 4-6 minutes
model_tfidf_3 = models.TfidfModel(bow, smartirs="lpu") # Logarithmic term frequency weighting, probabilistic document frequency weighting 
                                                        # and pivoted unique normalization
model_tfidf_2 = models.TfidfModel(bow, smartirs="lfc") # Logarithmic term frequency weighting, idf document frequency weighting 
                                                         # and cosine normalization
model_tfidf_5 = models.TfidfModel(bow, smartirs="ltc") # Logarithmic term frequency weighting, zero-corrected idf document frequency weighting 
                                                         # and cosine normalization
model_tfidf_4 = models.TfidfModel(bow, smartirs="nnn") # Raw term frequency weighting, none document frequency weighting 
                                                         # and none normalization

## Word2Vec

After creating several TFIDF models, we wanted to try the Word2Vec model.

In [ ]:
REPLACE_NO_SPACE = re.compile("(\&)|(\%)|(\$)|(\€)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\⁰)|(\•)|(\\')")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def clean(doc):
    """
    Basic function to clean a document being a document a string with words separated by blank spaces.
    """
    doc = REPLACE_NO_SPACE.sub(NO_SPACE, doc.lower())
    doc = REPLACE_WITH_SPACE.sub(SPACE, doc)
    stopset = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    tokens = wordpunct_tokenize(doc)
    clean = [token.lower() for token in tokens if token not in stopset and len(token) > 2 and not token.isnumeric()]
    final = [stemmer.stem(token) for token in clean]
    return " ".join(final)

### Creation

The first thing that we need to do is to create our Word2Vec model. To do that, we need to obtain our dataset (documents and queries).

The following cells create the dataset for our model.

In [ ]:
docs = pd.DataFrame((line for line in open("docs.txt")), columns = ["preprocess"])

In [ ]:
queries_w2v = queries.copy()
queries_w2v["preprocess"] = queries_w2v["query"].apply(lambda x: clean(x))

In [ ]:
combined = pd.concat((docs["preprocess"], queries_w2v["preprocess"])).sample(frac=1).reset_index(drop=True)

Once we have our dataset, it´s time to create and train our model. 
* **Size**: this is the size of the word embeddings. We chose this value after looking in some papers that suggested using a size between 200-500 for large corpus (as ours).
* **Workers**: if you execute this code, please change this number to maximum number of threads of your computer (cores x 2 if you have hyper threading)

**IMPORTANT** The following cell takes 10 minutes to finish in a computer with a Ryzen 5 3600. Remember this before executing it.

In [ ]:
# 10-20 minutes
w2v_model = models.Word2Vec([i.split() for i in combined], size = 300, workers=12)

In [ ]:
len(w2v_model.wv.vocab) # We check the size of our vocabulary, having more than 300k unique words.

### Saving

After creating our model, we can save it for later use in other applications.

In [ ]:
word_vectors = w2v_model.wv
word_vectors.save("word2vec.wordvectors")

In [ ]:
w2v_model.save("word2vec.model")

In [ ]:
del w2v_model # We delete the model since we only need the word_vectors

### Loading

We load our word_vectors.

In [ ]:
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load("word2vec.wordvectors", mmap='r')

### Getting embedding

In [ ]:
def get_embedding_w2v(doc_tokens):
    """
    Basic function to obtain the vector representation of a given document (list of tokens).
    
    """
    embeddings = []
    if len(doc_tokens) < 1: # If the length of our document is 0 we return a vector of zeros with size 300 (remember this is the vector size used when creating the model)
        return np.zeros(300)
    else:
        for token in doc_tokens:
            if token in word_vectors.vocab: # if the word is in our vocabulary, we get its vector
                embeddings.append(word_vectors.word_vec(token))
            else:
                embeddings.append(np.random.rand(300)) # if it´s not in our vocabulary a random vector is used
        return np.mean(embeddings, axis = 0) # the vector representation of a document is the mean of its word´s vectors

### Queries embeddings

We compute the vector representations of our queries.

In [ ]:
queries_w2v["embeddings"] = queries_w2v["preprocess"].apply(lambda x: get_embedding_w2v(x.split()))
queries_w2v.head()

### Documents embeddings

In order to reduce the amount of embeddings to be computed, we filter the documents and we only save the ones that have some query performed on it and appear in metadata.
Then, we compute the embedding for each document, so as to have it precomputed and then, when computing the similarity, just taking this value.

In [ ]:
titles_w2v = titles.copy()
titles_w2v = titles_w2v.reset_index(drop=True)

In [ ]:
docs = titles_w2v.join(docs)
docs.head()

In [ ]:
# With this filter, we only use documents that appear in both judgements and metadata files, i.e, documents that have queries related.
docs["sha"] = docs["id"]
inCorpus = metadata[metadata["sha"].isin(docs["sha"])]["cord_uid"]
inJudgements = inCorpus[inCorpus.isin(judgements["cord_uid"])]
filtered = metadata.loc[metadata['cord_uid'].isin(inJudgements)]["sha"].values
docs_filtered = docs.loc[docs["id"].isin(filtered)]
docs_filtered.head()

In [ ]:
# 5-10 minutes
docs_filtered["embeddings"] = docs_filtered["preprocess"].apply(lambda x: get_embedding_w2v(x.split())) 

In [ ]:
docs_filtered.head()

### Launch query in W2V Model

As before with the TFIDF model, we create two functions to launch queries with our W2V model.

We will show you the results for query 2.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
i = 2

def launch_query_w2v(query_name, queries, docs_filtered, verbose=0):
    """
    General function that allow us to launch queries to our W2V model.
    
    Parameter:
    query_name: the name of the query
    queries: dataframe of the queries
    docs_filtered: dataframe of filtered documents with their vector representation (the one explained before)
    
    Returns:
    
    It returns the ranking.
    
    """
    query_name = query_name
    # We obtain the vector of the query
    query_vector = queries[queries["query"] == query_name]["embeddings"].values[0]
    docs_aux = docs_filtered.copy()
    # We compute the cosine similarity of the query´s vector with the documents vectors
    docs_aux["score"] = docs_aux["embeddings"].apply(
        lambda x: cosine_similarity(np.array(query_vector).reshape(1,-1), np.array(x).reshape(1,-1)).item())
    # We order the docs by this similarity
    docs_aux = docs_aux.sort_values(by="score", ascending=False)
    doc_pos = docs_aux.index.values
    rel_score = docs_aux["score"].values
    ranking = [z for z in zip(doc_pos, rel_score)]
    if verbose:
        print("Query ==> "+query_name)
        for i in range(5):
            score = docs_aux.iloc[i]["score"]
            title = docs_aux.iloc[i]["title"] if docs_aux.iloc[i]["title"]!="" else "No title available"
            print(f"[Score = {score}] "+ title)
    return ranking, "Just to work"

def launch_w2v(function, queries, docs_filtered):
    """
    Again, function that allow us to reuse our code for evaluation.
    """
    def h(query_name):
        return function(query_name, queries, docs_filtered)
    return h

Execution of query 2 with its results. Notice that now the scores are higher.

In [ ]:
ranking, _ = launch_query_w2v(queries_w2v.iloc[1][0], queries_w2v, docs_filtered, verbose=1)

## Fast text

Fast text is a model based on the w2v model and the key difference among them is the use of n-grams. w2v treats each word as an atomic entity, so it generates a vector for each word. On the other hand, fastText considers each word as composed of character n-grams (the vector of the word is composed by the sum of its character n-grams - e.g. apple with smallest n-gram of 3 and largest n-gram of 6: ['<ap','app','appl','apple','apple>','ppl','pple','pple>','ple','ple>','le>']). 

### Creation

In order to create the model, we use the same dataset as for w2v.

**IMPORTANT** The fast text model takes around 30 minutes to be computed, be careful before running the next cell.

In [ ]:
# 20-30 minutes
fasttext_model = models.FastText([i.split() for i in combined], size = 50, workers=12)

### Saving

After creating our model, we can save it for later use in other applications.

In [ ]:
word_vectors_ft = fasttext_model.wv
word_vectors_ft.save("fasttext_model.wordvectors")

In [ ]:
fasttext_model.save("fasttext_model.model")

We delete the model since we only need the word_vectors.

In [ ]:
del fasttext_model

### Loading

We load our word_vectors for the fast text model.

In [ ]:
from gensim.models import KeyedVectors
word_vectors_ft = KeyedVectors.load("fasttext_model.wordvectors", mmap='r')

### Getting embeddings

The method *get_embedding_ft* obtain the vector representation of a given document. This document is represented as a list of tokens.

In [ ]:
def get_embedding_ft(doc_tokens):
    """
    Basic function to obtain the vector representation of a given document (list of tokens).
    
    """
    embeddings = []
    if len(doc_tokens) < 1: # If the length of our document is 0 we return a vector of zeros with size 300 (remember this is the vector size used when creating the model)
        return np.zeros(50)
    else:
        for token in doc_tokens:
            if token in word_vectors_ft.vocab: # if the word is in our vocabulary, we get its vector
                embeddings.append(word_vectors_ft.word_vec(token))
            else:
                embeddings.append(np.random.rand(50))
                #key = word_vectors_ft.most_similar(token)[0][0]
                #embeddings.append(word_vectors_ft.word_vec(key)) # if it´s not in our vocabulary a random vector is used
        return np.mean(embeddings, axis = 0) # the vector representation of a document is the mean of its word´s vectors

### Queries embeddings

We compute the vector representations of our queries (represented as a list of tokens).

In [ ]:
queries_ft = queries.copy()
queries_ft["preprocess"] = queries_ft["query"].apply(lambda x: clean(x))
queries_ft["embeddings"] = queries_ft["preprocess"].apply(lambda x: get_embedding_ft(x.split()))
queries_ft.head()

### Documents embeddings

As in w2v, in order to reduce the amount of embeddings to be computed, we filter the documents and we only save the ones that have some query performed on it and appear in metadata. Then, we compute the embedding for each document, so as to have it precomputed and then, when computing the similarity, just taking this value.

In [ ]:
titles_ft = titles.copy()
titles_ft = titles_ft.reset_index(drop=True)

In [ ]:
docs = pd.DataFrame((line for line in open("docs.txt")), columns = ["preprocess"])
docs_ft = titles_ft.join(docs)
docs_ft.head()

In [ ]:
# With this filter, we only use documents that appear in both judgements and metadata files, i.e, documents that have queries related.
docs_ft["sha"] = docs_ft["id"]
inCorpus_ft = metadata[metadata["sha"].isin(docs_ft["sha"])]["cord_uid"]
inJudgements_tf = inCorpus_ft[inCorpus_ft.isin(judgements["cord_uid"])]
filtered_ft = metadata.loc[metadata['cord_uid'].isin(inJudgements_tf)]["sha"].values
docs_filtered_ft = docs_ft.loc[docs_ft["id"].isin(filtered_ft)]
docs_filtered_ft.head()

In [ ]:
# 5-10 minutes
docs_filtered_ft["embeddings"] = docs_filtered_ft["preprocess"].apply(lambda x: get_embedding_ft(x.split())) 

In [ ]:
docs_filtered_ft.head()

### Launch query in FastText Model

As in w2v, we need to create two functions to launch queries with out FastText model. However, these functions are the same as in w2c so we will reuse them.

Execution of query 2 with its results. Notice that now the scores are higher than in TFIDF and w2v models.

In [ ]:
ranking_ft, _ = launch_query_w2v(queries_ft.iloc[1][0], queries_ft, docs_filtered_ft, verbose=1)

## Latent Semantic Indexing

LSI (Deerwester et al., 1990) is a retrieval model based on distributional semantics, that assumes that words tend to have similar meanings if they are used in the same context. This model uses Singular Value Decomposition. 

### Model creation

As always, we first create the model. To do that, we need to compute the corpus in tfidf format. Once it's done, we create the model using 100 as number of topics. The related literature suggested using a num_topics from hundreds to thousand, so we chose 100 since for this value we already obtain good performance and the execution time isn't too big.

**IMPORTANT** The following cell takes 5 minutes more or less to be executed.

In [ ]:
# 7-10 minutes
corpus_tfidf = model_tfidf[bow]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics = 100)
lsi.save("model_lsi.lsi")

### Similarities matrix

In [ ]:
# 3-5 minutes
index_lsi = Similarity(None, lsi[corpus_tfidf], num_features=len(dictionary))
index_lsi.save("covid19-lsi.index")

### Launching queries

The following cell contains the code needed to launch a query with LSI model.

In [ ]:
def launch_query_lsi(model, dictionary, bow, index, query, titles, verbose = 0):
    """
    Given a specific query, it returns the ranking of documents.
    
    Parameters
    ----------
    model : tfidf model
    dictionary: dictionary created
    bow: bag of words
    index: similarities matrix
    query: specific query
    titles: dataframe of the titles
    verbose: flag for printing messages
    
    Returns
    -------
    ranking: ranking for the query in this format: [(doc_position, score), (doc_position, score), ... ]
    similarities: similarities matrix

    """
    stopset = set(stopwords.words("english"))
    index = index
    pq = preprocess_query_tfidf(query, stopset)
    vq = dictionary.doc2bow(pq)
    veclsi = model[model_tfidf[vq]]
    sim = index[veclsi]
    ranking = sorted(enumerate(sim), key=itemgetter(1), reverse=True)
    if verbose:
        print("Query ==> "+query)
        for doc, score in ranking[:5]:
            print("[ Score = " + "%.3f" % round(score,3) + " ] " + titles['title'].iloc[doc])
    return ranking, sim

Launching query *origin coronavirus*...

In [ ]:
ranking, _ = launch_query_lsi(lsi, dictionary, bow, index_lsi, "origin coronavirus", titles, verbose = 1)

# Results

So, after explaining how we implemented the models, it's turn to evaluate them.

We will consider the metrics explained before: precision, recall, mean average precision, P@5, P@10 and R-Precision.

The following cell performs all the evaluation:
* We print the P@5, P@10 and MAVP of each single document.
* We plot a graph comparing the precision and recall of all the models tested.
* We plot a graph comparing the difference of the R-precision of the best two models for each single query. If the values are positive mean that R-precision of model A was greater than model's B R-Precision. Negative values mean that that R-precision of model B was greater than model's A R-Precision.

The evaluation procedure was created following the indications in (Baeza and Ribeiro, 2011).

In [ ]:
# 13-20 minutes
recall = np.arange(0,1.1, 0.1)
modelos_tfidf = [{"name":"TFIDF (nfc)", "term": "raw", "document": "idf", "normalization": "cosine", "model": model_tfidf, "color": "r"},
          {"name":"TFIDF (lfc)", "term": "logarithmic", "document": "idf", "normalization": "cosine", 
           "model": model_tfidf_2, "color": "g"},
           {"name":"TFIDF (lpu)", "term": "logarithmic", "document": "probabilistic", "normalization": "pivoted unique", 
           "model": model_tfidf_3, "color": "b"},
           {"name":"TFIDF (nnn)", "term": "raw", "document": "none", "normalization": "none", 
           "model": model_tfidf_4, "color": "c"},
           {"name":"TFIDF (ltc)", "term": "logarithmic", "document": "zero-corrected idf", "normalization": "cosine", 
           "model": model_tfidf_5, "color": "m"}
         ]

modelos_w2v = [{"name": "Word2Vec", "algorithm": "Continuous Bag of Words", "vector": 300, 
                "window": "5 (default)", "negative": "5 (default)", "epochs": "5 (default)", "color":"orange"}
              ]

modelos_ft = [{"name":"FastText",  "algorithm": "Continuous Bag of Words", "vector": 50, 
                "window": "5 (default)", "negative": "5 (default)", "epochs": "5 (default)", "color":"y"}]

modelos_lsi = [{"name": "Latent Semantic Index", "vector": 100, "version": "tfidf", "color":"k", "model": lsi}]
               
r_precisions = []
precisions_5 = []
precisions_10 = []

print("###################################################################################")
print("Evaluating performance of our models...")
plt.figure(figsize=(6,6))
plt.title("Precision vs Recall standardized")
plt.axis([0, 1, 0, 1])
plt.xlabel("Recall")
plt.ylabel("Precision")

for model in modelos_tfidf:
    name = model["name"]
    term = model["term"]
    document = model["document"]
    normalization = model["normalization"]
    color = model["color"]
    print("-------------------------------------------------------------------------------------")
    print(f"Evaluating {name}: {term} term frequency weighing, {document} document frequency weighting, {normalization} normalization")
    print("Computing statistics...")
    data_tfidf = get_statistics(launch(launch_query_tfidf,model["model"], dictionary, bow, similarities, titles.copy()), queries.copy(), titles.copy(), metadata.copy(), judgements.copy())
    print("Statistics computed...")
    precision_tfidf = data_tfidf["precision"]
    recall = data_tfidf["recall"]
    mavp_tfidf = data_tfidf["mavp"]
    precisions_5.append(np.array(data_tfidf["p_5"]).mean())
    precisions_10.append(np.array(data_tfidf["p_10"]).mean())
    r_precisions.append(data_tfidf["r-precision"])
    print(f"Mean Average Precision of this model ==> {mavp_tfidf}")
    p_5 = np.array(data_tfidf["p_5"]).mean()
    p_10 = np.array(data_tfidf["p_10"]).mean()
    print(f"Mean P@5 of this model ==> {p_5}")
    print(f"Mean P@10 of this model ==> {p_10}")

    plt.plot(recall, precision_tfidf, f"{color}-o", label=name)

    
for model in modelos_w2v:
    name = model["name"]
    vector = model["vector"]
    window = model["window"]
    negative = model["negative"]
    algorithm = model["algorithm"]
    epochs = model["epochs"]
    color = model["color"]
    print("-------------------------------------------------------------------------------------")
    print(f"Evaluating {name}: \n\tTraining algorithm = {algorithm}\n\tVector size = {vector}\n\tDistance predicted and current word = {window}\n\tNegative sampling = {negative}\n\tEpochs = {epochs}")
    print("Computing statistics...")
    data_w2v = get_statistics(launch_w2v(launch_query_w2v, queries_w2v.copy(), docs_filtered.copy()), queries_w2v.copy(), titles_w2v.copy(), metadata.copy(), judgements.copy())
    print("Statistics computed...")
    precision_w2v = data_w2v["precision"]
    recall = data_w2v["recall"]
    mavp_w2v = data_w2v["mavp"]
    r_precisions.append(data_w2v["r-precision"])
    precisions_5.append(np.array(data_w2v["p_5"]).mean())
    precisions_10.append(np.array(data_w2v["p_10"]).mean())
    print(f"Mean Average Precision of this model ==> {mavp_w2v}")
    p_5 = np.array(data_w2v["p_5"]).mean()
    p_10 = np.array(data_w2v["p_10"]).mean()
    print(f"Mean P@5 of this model ==> {p_5}")
    print(f"Mean P@10 of this model ==> {p_10}")

    plt.plot(recall, precision_w2v, "-o", color = color, label=name)

    
for model in modelos_ft:
    name = model["name"]
    vector = model["vector"]
    window = model["window"]
    negative = model["negative"]
    algorithm = model["algorithm"]
    epochs = model["epochs"]
    color = model["color"]
    print("-------------------------------------------------------------------------------------")
    print(f"Evaluating {name}: \n\tTraining algorithm = {algorithm}\n\tVector size = {vector}\n\tDistance predicted and current word = {window}\n\tNegative sampling = {negative}\n\tEpochs = {epochs}")
    print("Computing statistics...")
    data_ft = get_statistics(launch_w2v(launch_query_w2v, queries_ft.copy(), docs_filtered_ft.copy()), queries_ft.copy(), titles_ft.copy(), metadata.copy(), judgements.copy())
    print("Statistics computed...")
    precision_ft = data_ft["precision"]
    recall = data_ft["recall"]
    mavp_ft = data_ft["mavp"]
    r_precisions.append(data_ft["r-precision"])
    precisions_5.append(np.array(data_ft["p_5"]).mean())
    precisions_10.append(np.array(data_ft["p_10"]).mean())
    print(f"Mean Average Precision of this model ==> {mavp_ft}")
    p_5 = np.array(data_ft["p_5"]).mean()
    p_10 = np.array(data_ft["p_10"]).mean()
    print(f"Mean P@5 of this model ==> {p_5}")
    print(f"Mean P@10 of this model ==> {p_10}")

    plt.plot(recall, precision_ft, f"{color}-o", label=name)
    
for model in modelos_lsi:
    name = model["name"]
    vector = model["vector"]
    version = model["version"]
    color = model["color"]
    print("-------------------------------------------------------------------------------------")
    print(f"Evaluating {name}: \n\tVector size = {vector}\n\tCorpus version = {version}")
    print("Computing statistics...")
    data_lsi = get_statistics(launch(launch_query_lsi,model["model"], dictionary, model_tfidf[bow], index_lsi, titles.copy()), queries.copy(), titles.copy(), metadata.copy(), judgements.copy())
    print("Statistics computed...")
    precision_lsi = data_lsi["precision"]
    recall = data_lsi["recall"]
    mavp_lsi = data_lsi["mavp"]
    r_precisions.append(data_lsi["r-precision"])
    precisions_5.append(np.array(data_lsi["p_5"]).mean())
    precisions_10.append(np.array(data_lsi["p_10"]).mean())
    print(f"Mean Average Precision of this model ==> {mavp_lsi}")
    p_5 = np.array(data_lsi["p_5"]).mean()
    p_10 = np.array(data_lsi["p_10"]).mean()
    print(f"Mean P@5 of this model ==> {p_5}")
    print(f"Mean P@10 of this model ==> {p_10}")

    plt.plot(recall, precision_lsi, f"{color}-o", label=name)

plt.legend()
plt.show()
print("-------------------------------------------------------------------------------------")
print("Comparing our models: W2V vs LSI")
r_precision = np.array(r_precisions[-3]) - np.array(r_precisions[-1])
# the histogram of the data
plt.figure(figsize=(10,6))
plt.bar(np.arange(1,51,1),r_precision)
plt.xlabel('Query number')
plt.ylabel('R-Precision W2V/LSI')
plt.title('Comparison of R-Precisions')
plt.grid(True)
plt.show()
print("-------------------------------------------------------------------------------------")
print("Evalution finished.")
print("###################################################################################")

# Conclusions



This work aims to retrieve relevant COVID documents given a query and, hence, different models have been implemented and tested. The characteristics of these models are the following:

* TFIDF. The term frequency (tf), document frequency (idf) and document normalization can be computed using different metrics. In this work, different combinations have been tested and the one with better performance is lpu: logarithm term frequency weighting, probabilist document frequency weighting and pivoted unique document normalization.
* w2v (word2vec). This model computes the weights of the first layer of a neural network with one unique hiden layer. It performs better than any of the TFIDF models (r-precision, P@5 and P@10 improve their values).
* fastText. It is based on the w2v model and the key difference among them is the use of n-grams. This model performs worse than w2v and take more time to train the model. Consequently, if the system will run in a real environment (not a learning one), we will exclude it from the possible retrieve models for the search engine. However, recent literature (Tiun et al.) reports that fastTest has better performance than w2v. This let us conclude that the performance of each model varies significantly among corpora. In comparison with TFIDF, we think that the improvement in the evaluation metrics is not worth it compared to the model training time.
* Latent Semantic Indexing (LSI). This model considers the semantic meaning of the words: words that are in the same context tend to appear together. Regarding the mean average precision and the r-precisions, this model performs better than w2v. However, w2v achieves a higher value for P@5 and P@10. 

The two models that have a better performance are w2v and LSI. If we observe the graphic that compares the r-precision of both models in all models, we can see that, in general, LSI is better. However, P@5 and P@10 are higher in w2c and this suggest:

* If we want to retrieve just a few results: better performance w2v (a higher value for P@5 and P@10).
* If we want to retrieve all the results in the ranking: better performance LSI (a higher value for mean average precision). 

To conclude, we want to mention that there isn't any perfect model and, in this case, we select LSI and w2v. As a future line of work, we have seen in literature that results from different models can be integrated to retrieve a combined ranking. Additionally, doc2vec model will be tested in order to evaluate its performance: there is no need in getting the embeddings (reduces computational load).

# References

Baeza-Yates, R., & Ribeiro-Neto, B. (2011). Modern information retrieval: the concepts and technology behind search. Choice Reviews Online, 48(12), 48-6950-48–6950. https://doi.org/10.5860/choice.48-6950

Deerwester, S., Dumais, S. T., Furnas, G. W., Landauer, T. K., & Harshman, R. (1990). Indexing by latent semantic analysis. In Journal of the American Society for Information Science (Vol. 41, Issue 6). https://doi.org/10.1002/(SICI)1097-4571(199009)41:6<391::AID-ASI1>3.0.CO;2-9

Mikolov, T., Sutskever, I., Chen, K., Corrado, G., & Dean, J. (2013). Distributed representations ofwords and phrases and their compositionality. Advances in Neural Information Processing Systems. http://arxiv.org/abs/1310.4546

Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. In 1st International Conference on Learning Representations, ICLR 2013 - Workshop Track Proceedings. http://ronan.collobert.com/senna/

Tiun, S., U. A. Mokhtar, S. H. Bakar, and S. Saad. (2020). Classification of functional and non-functional requirement in software requirement using Word2vec and fast Text. In Journal of Physics: Conference Series, vol. 1529, no. 4, p. 042077. https://doi.org/10.1088%2F1742-6596%2F1529%2F4%2F042077